In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from pymongo import MongoClient

import os
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

import pickle
import re

In [3]:
client=MongoClient()
db = client['proj_4_gutenberg']
collection = db['gutenberg_books']

In [4]:
def book_to_mongo(book_num):

    book_details = {}
    book_details['_id']  = str(book_num)
    driver = webdriver.Chrome(chromedriver)
    try:
        driver.get('https://www.gutenberg.org/ebooks/%s' % book_num)
        book_text = driver.find_element_by_link_text('Read this book online: HTML')
        #     book_text=driver.find_element_by_xpath('//*[@id="download"]/div/table/tbody/tr[8]/td[2]/a')
        book_text.click()
        time.sleep(2)
        pre_mat = driver.find_elements_by_tag_name('pre')
      
        title_search = re.compile('Title:(.*?)\\n')
        title = title_search.search(str(pre_mat[0].text))
        print('hello')
        title = title.group(0)
        
        just_title = re.compile(':(.*)')
        book_title = just_title.search(title)
        title = book_title.group(1).strip()
        print(title)
        book_details['title'] = title


        author_search = re.compile('Author:(.*?)\\n')
        author = author_search.search(str(pre_mat[0].text))
        author = str(author.group(0))
        just_name = re.compile(':(.*)')
        author_name = just_name.search(author)
        author = author_name.group(1).strip()
        book_details['author'] = author


        book_text = driver.find_elements_by_tag_name('p')
        text_for_dict = ''
        for x in range(len(book_text)):
            text_for_dict += (book_text[x].text)
        book_details['text'] = text_for_dict

        print(book_details)
        driver.close()
        
        collection.insert_one(book_details)
    except:
        driver.close()
    print(book_details)
    return book_details

In [6]:
book_to_mongo(25344)

hello
The Scarlet Letter
{'_id': '25344', 'title': 'The Scarlet Letter', 'author': 'Nathaniel Hawthorne', 'text': "BYNATHANIEL HAWTHORNE.Illustrated.BOSTON:\nJAMES R. OSGOOD AND COMPANY,\nLate Ticknor & Fields, and Fields, Osgood, & Co.\n1878.Copyright, 1850 and 1877.\nBy NATHANIEL HAWTHORNE and JAMES R. OSGOOD & CO.All rights reserved.\nOctober 22, 1874.UCH to the author's surprise, and (if he may say so without additional offence) considerably to his amusement, he finds that his sketch of official life, introductory to The Scarlet Letter, has created an unprecedented excitement in the respectable community immediately around him. It could hardly have been more violent, indeed, had he burned down the Custom-House, and quenched its last smoking ember in the blood of a certain venerable personage, against whom he is supposed to cherish a peculiar malevolence. As the public disapprobation would weigh very heavily on him, were he conscious of deserving it, the author begs leave to say, th

{'_id': '25344',
 'title': 'The Scarlet Letter',
 'author': 'Nathaniel Hawthorne',
 'text': "BYNATHANIEL HAWTHORNE.Illustrated.BOSTON:\nJAMES R. OSGOOD AND COMPANY,\nLate Ticknor & Fields, and Fields, Osgood, & Co.\n1878.Copyright, 1850 and 1877.\nBy NATHANIEL HAWTHORNE and JAMES R. OSGOOD & CO.All rights reserved.\nOctober 22, 1874.UCH to the author's surprise, and (if he may say so without additional offence) considerably to his amusement, he finds that his sketch of official life, introductory to The Scarlet Letter, has created an unprecedented excitement in the respectable community immediately around him. It could hardly have been more violent, indeed, had he burned down the Custom-House, and quenched its last smoking ember in the blood of a certain venerable personage, against whom he is supposed to cherish a peculiar malevolence. As the public disapprobation would weigh very heavily on him, were he conscious of deserving it, the author begs leave to say, that he has carefully re

In [12]:
cursor = collection.find({}, {'_id':1, 'title':1, 'author':1})
list(cursor)

[{'_id': '1342', 'title': 'Pride and Prejudice', 'author': 'Jane Austen'},
 {'_id': '84',
  'title': 'Frankenstein',
  'author': 'Mary Wollstonecraft (Godwin) Shelley'},
 {'_id': '219', 'title': 'Heart of Darkness', 'author': 'Joseph Conrad'},
 {'_id': '1080', 'title': 'A Modest Proposal', 'author': 'Jonathan Swift'},
 {'_id': '98', 'title': 'A Tale of Two Cities', 'author': 'Charles Dickens'},
 {'_id': '15399',
  'title': 'The Importance of Being Earnest',
  'author': 'Oscar Wilde'},
 {'_id': '76',
  'title': "Alice's Adventures in Wonderland",
  'author': 'Lewis Carroll'},
 {'_id': '2148',
  'title': 'Moby Dick; or The Whale',
  'author': 'Herman Melville'},
 {'_id': '58915', 'title': 'Voyage to Jupiter', 'author': 'David Morrison'},
 {'_id': '160',
  'title': 'The Adventures of Sherlock Holmes',
  'author': 'Arthur Conan Doyle'},
 {'_id': '2097',
  'title': 'The Souls of Black Folk',
  'author': 'W. E. B. Du Bois'},
 {'_id': '5200', 'title': 'Metamorphosis', 'author': 'Franz Kafka'}

In [43]:
# collection.delete_one({'_id':'1251'})


## Multiple Books

In [10]:
# books_to_add = [97,35879, 32579,43048]
books_to_add = [964]

In [11]:
for book in books_to_add:
    book_to_mongo(book)

hello
The Merry Adventures of Robin Hood
{'_id': '964', 'title': 'The Merry Adventures of Robin Hood', 'author': 'Howard Pyle', 'text': 'You who so plod amid serious things that you feel it shame to give yourself up even for a few short moments to mirth and joyousness in the land of Fancy; you who think that life hath nought to do with innocent laughter that can harm no one; these pages are not for you. Clap to the leaves and go no farther than this, for I tell you plainly that if you go farther you will be scandalized by seeing good, sober folks of real history so frisk and caper in gay colors and motley that you would not know them but for the names tagged to them. Here is a stout, lusty fellow with a quick temper, yet none so ill for all that, who goes by the name of Henry II. Here is a fair, gentle lady before whom all the others bow and call her Queen Eleanor. Here is a fat rogue of a fellow, dressed up in rich robes of a clerical kind, that all the good folk call my Lord Bishop o

In [ ]:
to_add = [15399,12354,26356,15044]

In [44]:
for x in cursor:
    print(x['text'])

## Attaching _ID

Because I initially loaded the books without an id, I had to rewrite the ids to match those listed by project Gutenberg.  The code below does that.

In [ ]:
import re

In [57]:
book_num_title = []
with open('../data/raw/handpicked/html_fromtop100.html', 'r') as html_f:
        book_num = re.compile('/\d*(?=")')
        book_title = re.compile('>\w.{10}')
#         book_num = re.compile('82')
        for line in html_f:
            num = book_num.search(line)
            num = (num[0][1:])
            print(num)
            
            title_frag = book_title.search(line)
            try:
                book_num_title.append([title_frag[0][1:], num])
            except:
                print(line)

1342
84
219
1080
2542
98
844
11
2701
58915
1661
408
76
5200
74
1952
2554
160
58912
2600
43
2591
345
1260
16328
6130
58914
174
25344
205
1184
1322
58911
1400
16
2814
23
3825
158
30254
5740
4300
2500
1232
46
120
3207
58913
768
8800
3600
20228
135
1934
1497
244
34901
55
33283
203
45502
1998
140
28054
2680
58900
161
786
20203
100
1251
61
19942
521
7370
4363
35
15399
121
30360
514
113
41
2852
375
851
58909
10676
730
1250
209
10
829
58899
2097
141
147
45
2148
27827


In [75]:
# for book in books_to_load:
#     for num_title in book_num_title:
#         if num_title[0][:5] in (book['title']):
#             book['_id'] = num_title[1]

In [76]:
# for book in books_to_load:
#     try:
#         print(book['title'], book['_id'])
#     except:
#         print(book['title'])

Pride and Prejudice 1342
Frankenstein 84
Heart of Darkness 219
A Modest Proposal 1080
A Doll's House
A Tale of Two Cities 98
The Importance of Being Earnest 15399
Alice's Adventures in Wonderland 76
Moby Dick; or The Whale 2148
Voyage to Jupiter 58915
The Adventures of Sherlock Holmes 160
The Souls of Black Folk 2097
Adventures of Huckleberry Finn, Complete 76
Metamorphosis 5200
The Adventures of Tom Sawyer, Complete 160
The Yellow Wallpaper 1952
Crime and Punishment
The Awakening and Selected Short Stories 160
The Earth Quarter 58912
War and Peace 2600
The Strange Case Of Dr. Jekyll And Mr. Hyde 2097
Grimms' Fairy Tales 2591
Dracula 345
Jane Eyre 1260
Beowulf 16328
The Iliad of Homer 15399
The White Eagle of Poland 2148
The Picture of Dorian Gray 174
The Scarlet Letter 2097
Walden, and On The Duty Of Civil Disobedience 8800
The Count of Monte Cristo 100
Leaves of Grass 1322
Autographs for Freedom 58911
Great Expectations 1400
Peter Pan 16
Dubliners 2814
The Narrative of the Life of Fr

In [14]:
client.database_names()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  """Entry point for launching an IPython kernel.


['admin',
 'books',
 'challenge_14',
 'config',
 'events',
 'local',
 'outings',
 'proj_4_gutenberg',
 'project_4']

In [163]:
# client.db.project_4.drop()